In [ ]:
import cv2
import numpy as np

In [ ]:
import cv2
import numpy as np

In [ ]:


def find_marker_and_annotate(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to get a binary image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Approximate the contour to a circle
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

        # Check if the contour is circular
        area = cv2.contourArea(contour)
        if len(approx) > 5 and area > 100:  # Adjust parameters as needed
            # Compute the center of the contour
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])

                # Draw the contour and the center of the shape on the image
                cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
                cv2.circle(image, (cX, cY), 7, (255, 0, 0), -1)
                cv2.putText(image, "Center", (cX - 20, cY - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                return image, (cX, cY)

    return image, None


def calculate_3d_position(left_point, right_point, Q):
    """
    Calculate the 3D position of the marker.
    """
    # Compute disparity
    disparity = left_point[0] - right_point[0]

    # Generate 3D coordinates
    point_3d = cv2.reprojectImageTo3D(np.array([[disparity]]), Q)

    return point_3d[0][0]

def main():
    # Load calibration parameters (camera matrices, distortion coefficients, rectification matrices, projection matrices, Q matrix)
    # ...

    # Open video capture for both cameras
    cap_left = cv2.VideoCapture(0)  # Adjust the index
    cap_right = cv2.VideoCapture(1)  # Adjust the index

    while True:
        # Capture frames from both cameras
        ret_left, frame_left = cap_left.read()
        ret_right, frame_right = cap_right.read()

        if not ret_left or not ret_right:
            break

        # Find the marker in both images
        marker_left = find_marker(frame_left)
        marker_right = find_marker(frame_right)

        # Calculate the 3D position of the marker
        marker_3d_position = calculate_3d_position(marker_left, marker_right, Q)

        # Display the results
        print("3D Position: ", marker_3d_position)

        # Show the frames (optional)
        cv2.imshow("Left Frame", frame_left)
        cv2.imshow("Right Frame", frame_right)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap_left.release()
    cap_right.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [124]:
import cv2
import numpy as np

def find_marker_and_annotate(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to get a binary image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Approximate the contour to a circle
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

        # Check if the contour is circular
        area = cv2.contourArea(contour)
        if len(approx) > 5 and area > 100:  # Adjust parameters as needed
            # Compute the center of the contour
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])

                # Draw the contour and the center of the shape on the image
                cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
                cv2.circle(image, (cX, cY), 7, (255, 0, 0), -1)
                cv2.putText(image, "Center", (cX - 20, cY - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                return image, (cX, cY)

    return image, None


In [ ]:
image = cv2.imread('image0.jpg')  # Replace with actual image capture
annotated_image, center = find_marker_and_annotate(image)

if center is not None:
    print("Marker center:", center)
else:
    print("Marker not found")

# Display the annotated image
cv2.imshow("Annotated Image", annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Marker center: (284, 270)
